In [1]:
import os
import sys
sys.path.append('../../')
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import recs.path as path
from recs.clf import Classifier
from recs.backtest import get_backtest_answer, model_score_logger
from recs.util import readjson2dict, id2cat, save_model, save2json, recall_evaluate
from sklearn.model_selection import train_test_split

In [2]:
params = {
    "best_sell_n": 100,
    "n_estimators":250,
    "max_depth":3,
    "negative_sampling":True,
    "train_period": [201600, 202000],
    "best_sell_period":[201800 ,202000],
    "predict_period": [202000, 202100]
}

In [3]:
trans_buy_path = os.path.join(path.data_path, "trans_buy.pkl")
trans_buy = pd.read_pickle(trans_buy_path)

In [4]:
test = get_backtest_answer(trans_df=trans_buy, 
                           start=params["predict_period"][0]*100, 
                           end=params["predict_period"][1]*100)

100%|██████████| 28496/28496 [01:16<00:00, 370.09it/s]


In [5]:
best_sell_n = list(trans_buy[
    (trans_buy.buy_date > params['best_sell_period'][0]*100) & 
    (trans_buy.buy_date < params['best_sell_period'][1]*100)
].fund_id.value_counts().index)[:params["best_sell_n"]]
rec_bestsell = {}
for u in tqdm(test):
    rec_bestsell[u] = best_sell_n

recall_evaluate(rec_bestsell, test)

100%|██████████| 28496/28496 [00:00<00:00, 160720.45it/s]


{'recall@3': 0.035873444373555885,
 'recall@5': 0.047851969556067575,
 'recall@10': 0.05577968018029278,
 'recall@all': 0.36979117860666194}